In [46]:
!pip install pandas

**Let's import the data we need that's already been saved to the `formatted` directory.  We'll be saving the processed data to `checkpoints` folder**

In [47]:
# Import necessary libraries
import pandas as pd
import os

# Read the CSV file
# Read all motorcycle CSV files
csv_files = [f for f in os.listdir('../data/formatted/') if f.startswith('motorcycle') and f.endswith('.csv')]
dfs = []
for file in csv_files:
    df_temp = pd.read_csv(f'../data/formatted/{file}')
    dfs.append(df_temp)
    
# Combine all dataframes
df = pd.concat(dfs, ignore_index=True)

# Display first few rows to verify data loading
print("Dataset shape:", df.shape)
df.head()


Dataset shape: (3254, 23)


,Model Name,Reference Price (Yuan),Brand,Model,Production Method,Origin,Engine,Max Horsepower (Ps),Max Power/Speed (kW/rpm),Max Torque/Speed (N·m/rpm),...,Dimensions (LxWxH mm),Seat Height (mm),Curb Weight (kg),Fuel Tank Capacity (L),Max Speed (km/h),Official Average Fuel Consumption (L/100km),Measured Average Fuel Consumption (L/100km),Range (km),Warranty,Available Colors
0,钛极 NEXY+/未界,Currently no quotation available,titanium,pedal,domestic,NaN,single cylinder four stroke water-cooled 150cc,15.8,11.6/8500,NaN,...,1970x770x1150,765.0,136.0,8.0,105.0,-,NaN,-,NaN,NaN
1,钛极 BOHEMIA/波西米亚,Currently no quotation available,titanium,pedal,domestic,NaN,single cylinder four stroke water-cooled 150cc,15.8,11.6/8500,NaN,...,1920x740x1120,785.0,130.0,8.0,100.0,-,NaN,-,NaN,Titanium Extreme Silver
2,钛极 PHANTOM/飞腾,Currently no quotation available,titanium,pedal,domestic,NaN,single cylinder four stroke water-cooled 300cc,25.2,18.5/8250,NaN,...,1900x790x1200,775.0,150.0,11.0,118.0,-,NaN,-,NaN,Dark Night Black
3,钛极 HELLCAT/地狱猫,Currently no quotation available,titanium,street car,domestic production,NaN,inline dual cylinder water-cooled 250cc,32.6,24/9500,NaN,...,2100x805x1060,710.0,168.0,13.0,120.0,-,NaN,-,NaN,Titanium Extreme Silver
4,柴特 ZFR525C 复仇者,29900,Chaite,sports car,domestic production,Jiangmen Changhua Group Co.,inline dual cylinder four stroke water-cooled ...,53.8,39.6/8500,NaN,...,205x80x1150,790.0,199.0,15.0,195.0,3.6,NaN,400,NaN,"Ice White, Track Blue"


**Ok.  Looks like our data is all there but we have NaN values in some columns.  This will just be the first step as we have strange commas and other things in the data.**

**Removing special characters is a good first step.**

In [48]:
# Clean string data by removing leading/trailing whitespace and special characters
def clean_string(value):
    if pd.isna(value) or not isinstance(value, str):
        return value
    # Remove leading/trailing whitespace
    value = value.strip()
    # Remove leading/trailing commas, dots, and other common special characters
    value = value.strip('.,;:\"\'')
    # Replace multiple spaces with single space
    value = ' '.join(value.split())
    return value

# Apply cleaning to all string (object) columns
for column in df.select_dtypes(include=['object']).columns:
    df[column] = df[column].apply(clean_string)
    
    # Print a sample of changes made (first 5 changes)
    mask = df[column] != df[column].apply(lambda x: x if pd.isna(x) else x.strip())
    if mask.any():
        print(f"\nCleaning results for {column}:")
        changes = pd.DataFrame({
            'Before': df[column][mask],
            'After': df[column].apply(lambda x: x if pd.isna(x) else x.strip())[mask]
        })
        print(changes.head())

# Remove any rows that are completely empty (all values are NaN)
initial_rows = len(df)
df = df.dropna(how='all')
rows_removed = initial_rows - len(df)
if rows_removed > 0:
    print(f"\nRemoved {rows_removed} completely empty rows")

print("\nCleaning complete!")



Cleaning results for Reference Price (Yuan):
    Before After
37     NaN   NaN
119    NaN   NaN
120    NaN   NaN
122    NaN   NaN
123    NaN   NaN

Cleaning results for Brand:
    Before After
122    NaN   NaN
123    NaN   NaN
124    NaN   NaN
126    NaN   NaN
127    NaN   NaN

Cleaning results for Model:
    Before After
59     NaN   NaN
105    NaN   NaN
106    NaN   NaN
107    NaN   NaN
108    NaN   NaN

Cleaning results for Production Method:
    Before After
59     NaN   NaN
127    NaN   NaN
128    NaN   NaN
322    NaN   NaN
323    NaN   NaN

Cleaning results for Origin:
  Before After
0    NaN   NaN
1    NaN   NaN
2    NaN   NaN
3    NaN   NaN
5    NaN   NaN

Cleaning results for Engine:
   Before After
22    NaN   NaN
43    NaN   NaN
76    NaN   NaN
77    NaN   NaN
78    NaN   NaN

Cleaning results for Max Power/Speed (kW/rpm):
   Before After
22    NaN   NaN
43    NaN   NaN
62    NaN   NaN
76    NaN   NaN
77    NaN   NaN

Cleaning results for Transmission:
   Before After
0    

**Is there any data missing?**

In [49]:
# Get total number of rows in the dataset
print(f"\nTotal number of rows: {len(df)}")

# Calculate and display number of NaN values for each column
nan_counts = df.isna().sum()
print("\nNumber of NaN values per column:")
print(nan_counts)

# Calculate percentage of NaN values
nan_percentages = (df.isna().sum() / len(df)) * 100
print("\nPercentage of NaN values per column:")
print(nan_percentages.round(2))



Total number of rows: 3254

Number of NaN values per column:
Model Name                                        0
Reference Price (Yuan)                          533
Brand                                            37
Model                                           296
Production Method                               116
Origin                                         2204
Engine                                          488
Max Horsepower (Ps)                            1003
Max Power/Speed (kW/rpm)                        992
Max Torque/Speed (N·m/rpm)                     3254
Transmission                                    466
ABS                                             314
CBS                                               0
Dimensions (LxWxH mm)                           684
Seat Height (mm)                               1427
Curb Weight (kg)                               1104
Fuel Tank Capacity (L)                          453
Max Speed (km/h)                               1336
Of

**Wow looks like quite a bit is missing.  We'll drop columns with more than 90% missing values.**


In [50]:
# Drop columns with more than 90% NaN values
threshold = 0.9  # 90% threshold
columns_to_drop = nan_percentages[nan_percentages > 90].index
df = df.drop(columns=columns_to_drop)

# Remove rows where price is NaN
price_col = 'Reference Price (Yuan)'
initial_rows = len(df)
df = df.dropna(subset=[price_col])
rows_removed = initial_rows - len(df)
print(f"\nRemoved {rows_removed} rows with missing price values")

# Replace brand values less than 3 characters with 'Not Specified'
df.loc[df['Brand'].fillna('').str.len() < 3, 'Brand'] = 'Not Specified'

# Capitalize model values
df['Model'] = df['Model'].str.capitalize()

print("\nColumns dropped due to >90% missing values:")
print(columns_to_drop.tolist())

print("\nRemaining columns:")
print(df.columns.tolist())



Removed 533 rows with missing price values

Columns dropped due to >90% missing values:
['Max Torque/Speed (N·m/rpm)', 'Measured Average Fuel Consumption (L/100km)', 'Warranty']

Remaining columns:
['Model Name', 'Reference Price (Yuan)', 'Brand', 'Model', 'Production Method', 'Origin', 'Engine', 'Max Horsepower (Ps)', 'Max Power/Speed (kW/rpm)', 'Transmission', 'ABS', 'CBS', 'Dimensions (LxWxH mm)', 'Seat Height (mm)', 'Curb Weight (kg)', 'Fuel Tank Capacity (L)', 'Max Speed (km/h)', 'Official Average Fuel Consumption (L/100km)', 'Range (km)', 'Available Colors']


In [51]:
# Get detailed info about remaining columns
print("Detailed analysis of remaining features:")
for column in df.columns:
    nan_pct = (df[column].isna().sum() / len(df)) * 100
    unique_vals = df[column].nunique()
    dtype = df[column].dtype
    
    print(f"\n{column}:")
    print(f"Missing values: {nan_pct:.2f}%")
    print(f"Data type: {dtype}")
    print(f"Number of unique values: {unique_vals}")
    if dtype in ['object', 'category']:
        print("Top 5 most common values:")
        print(df[column].value_counts().head())
    elif dtype in ['int64', 'float64']:
        print(f"Mean: {df[column].mean():.2f}")
        print(f"Median: {df[column].median():.2f}")

Detailed analysis of remaining features:

Model Name:
Missing values: 0.00%
Data type: object
Number of unique values: 2699
Top 5 most common values:
Model Name
金城 草蜢 200 警用版    2
力箭 海盗800         2
力箭 海盗500         2
力箭 LJ250B        2
力箭 海盗300         2
Name: count, dtype: int64

Reference Price (Yuan):
Missing values: 0.00%
Data type: object
Number of unique values: 681
Top 5 most common values:
Reference Price (Yuan)
Currently no quotation available    788
7980                                 46
6980                                 41
8980                                 28
23800                                23
Name: count, dtype: int64

Brand:
Missing values: 0.00%
Data type: object
Number of unique values: 253
Top 5 most common values:
Brand
Haojue      108
Zongshen     95
BIMOTA       85
Benelli      77
Sanyang      75
Name: count, dtype: int64

Model:
Missing values: 7.83%
Data type: object
Number of unique values: 11
Top 5 most common values:
Model
Pedal                    1

**Columns with less than 10% missing values are left in place but the rest will need to be imputed.**

**Categorical columns will be imputed with 'Not Specified' and the rest will be imputed with the median.**

**We're also creating flag columns to track which values were imputed.**

In [52]:
# Create copy of dataframe for imputation
df_imputed = df.copy()

# 1. Identify price-related columns and tech spec columns
price_columns = [col for col in df_imputed.select_dtypes(include=['int64', 'float64']).columns 
                 if 'price' in col.lower()]
tech_spec_columns = [col for col in df_imputed.select_dtypes(include=['int64', 'float64']).columns 
                    if col not in price_columns]

# 2. Impute prices with median and create flag columns
for col in price_columns:
    # Create flag column before imputation
    df_imputed[f'{col}_is_imputed'] = df_imputed[col].isna()
    # Perform imputation
    df_imputed[col] = df_imputed[col].fillna(df_imputed[col].median())

# 3. Impute technical specifications with median and create flag columns
for col in tech_spec_columns:
    # Create flag column before imputation
    df_imputed[f'{col}_is_imputed'] = df_imputed[col].isna()
    # Perform imputation
    df_imputed[col] = df_imputed[col].fillna(df_imputed[col].median())

# 4. Impute categorical columns with 'Not Specified'
categorical_columns = df_imputed.select_dtypes(include=['object']).columns
for col in categorical_columns:
    df_imputed[col] = df_imputed[col].fillna('Not Specified')

# Verify results
print("\nRemaining missing values after imputation:")
print(df_imputed.isna().sum())

# Show the new flag columns
flag_columns = [col for col in df_imputed.columns if col.endswith('_is_imputed')]
print("\nNew imputation flag columns:")
print(flag_columns)

# Show sample of rows where values were imputed
for col in price_columns + tech_spec_columns:
    imputed_rows = df_imputed[df_imputed[f'{col}_is_imputed']]
    if len(imputed_rows) > 0:
        print(f"\nSample of imputed values for {col}:")
        print(imputed_rows[[col, f'{col}_is_imputed']].head())


Remaining missing values after imputation:
Model Name                                     0
Reference Price (Yuan)                         0
Brand                                          0
Model                                          0
Production Method                              0
Origin                                         0
Engine                                         0
Max Horsepower (Ps)                            0
Max Power/Speed (kW/rpm)                       0
Transmission                                   0
ABS                                            0
CBS                                            0
Dimensions (LxWxH mm)                          0
Seat Height (mm)                               0
Curb Weight (kg)                               0
Fuel Tank Capacity (L)                         0
Max Speed (km/h)                               0
Official Average Fuel Consumption (L/100km)    0
Range (km)                                     0
Available Colors         

In [53]:
df_imputed.head()

,Model Name,Reference Price (Yuan),Brand,Model,Production Method,Origin,Engine,Max Horsepower (Ps),Max Power/Speed (kW/rpm),Transmission,...,Fuel Tank Capacity (L),Max Speed (km/h),Official Average Fuel Consumption (L/100km),Range (km),Available Colors,Max Horsepower (Ps)_is_imputed,Seat Height (mm)_is_imputed,Curb Weight (kg)_is_imputed,Fuel Tank Capacity (L)_is_imputed,Max Speed (km/h)_is_imputed
0,钛极 NEXY+/未界,Currently no quotation available,titanium,Pedal,domestic,Not Specified,single cylinder four stroke water-cooled 150cc,15.8,11.6/8500,Not Specified,...,8.0,105.0,-,-,Not Specified,False,False,False,False,False
1,钛极 BOHEMIA/波西米亚,Currently no quotation available,titanium,Pedal,domestic,Not Specified,single cylinder four stroke water-cooled 150cc,15.8,11.6/8500,CVT continuously variable transmission automat...,...,8.0,100.0,-,-,Titanium Extreme Silver,False,False,False,False,False
2,钛极 PHANTOM/飞腾,Currently no quotation available,titanium,Pedal,domestic,Not Specified,single cylinder four stroke water-cooled 300cc,25.2,18.5/8250,CVT continuously variable transmission automat...,...,11.0,118.0,-,-,Dark Night Black,False,False,False,False,False
3,钛极 HELLCAT/地狱猫,Currently no quotation available,titanium,Street car,domestic production,Not Specified,inline dual cylinder water-cooled 250cc,32.6,24/9500,-,...,13.0,120.0,-,-,Titanium Extreme Silver,False,False,False,False,False
4,柴特 ZFR525C 复仇者,29900,Chaite,Sports car,domestic production,Jiangmen Changhua Group Co,inline dual cylinder four stroke water-cooled ...,53.8,39.6/8500,international 6-speed,...,15.0,195.0,3.6,400,"Ice White, Track Blue",False,False,False,False,False


In [54]:
# Replace '-' and empty values with appropriate defaults
for column in df_imputed.columns:
    # For numeric columns, replace with -1
    if df_imputed[column].dtype in ['int64', 'float64']:
        df_imputed[column] = df_imputed[column].replace('-', -1)
        df_imputed[column] = df_imputed[column].replace('', -1)
    else:
        # For categorical/string columns, replace with 'Not specified' 
        df_imputed[column] = df_imputed[column].replace('-', 'Not specified')
        df_imputed[column] = df_imputed[column].replace('', 'Not specified')

# Verify results
print("\nSample rows after replacing '-' and empty values:")
print(df_imputed.head())



Sample rows after replacing '-' and empty values:
        Model Name            Reference Price (Yuan)     Brand       Model  \
0      钛极 NEXY+/未界  Currently no quotation available  titanium       Pedal   
1  钛极 BOHEMIA/波西米亚  Currently no quotation available  titanium       Pedal   
2    钛极 PHANTOM/飞腾  Currently no quotation available  titanium       Pedal   
3   钛极 HELLCAT/地狱猫  Currently no quotation available  titanium  Street car   
4   柴特 ZFR525C 复仇者                             29900    Chaite  Sports car   

     Production Method                      Origin  \
0             domestic               Not Specified   
1             domestic               Not Specified   
2             domestic               Not Specified   
3  domestic production               Not Specified   
4  domestic production  Jiangmen Changhua Group Co   

                                              Engine  Max Horsepower (Ps)  \
0     single cylinder four stroke water-cooled 150cc                 15.8   
1

**We're now ready to save our preprocessed data to the `checkpoints` folder.**

In [55]:
# Save the preprocessed dataframe to checkpoints folder
df_imputed.to_csv('../data/checkpoints/imputed_full_motorcycle_data.csv', index=False)
print("Preprocessed data saved to checkpoints/preprocessed_data.csv")


Preprocessed data saved to checkpoints/preprocessed_data.csv


**Let's move on the next couple of data sheets.  While we can't iterate over such a number of files.  We can work with these manually.**

In [56]:
# Load the brochure motorcycle data
df_brochure = pd.read_csv('../data/formatted/brochure_motorcycle.csv')

# Drop columns with all missing values
df_brochure_cleaned = df_brochure.dropna(axis=1, how='all')

# Show info about dropped columns
dropped_cols = set(df_brochure.columns) - set(df_brochure_cleaned.columns)
print("\nColumns dropped due to all missing values:")
print(dropped_cols)

# Save the cleaned brochure dataframe
df_brochure_cleaned.to_csv('../data/checkpoints/full_brochure_motorcycle_data.csv', index=False)
print("\nCleaned brochure data saved to checkpoints/cleaned_brochure_motorcycle_data.csv")




Columns dropped due to all missing values:
{'Unnamed: 36', 'Unnamed: 75', 'Unnamed: 24', 'Unnamed: 64', 'Unnamed: 4', 'Unnamed: 38', 'Container loading （CKD', 'Unnamed: 76', 'Economical Fuel', 'Unnamed: 15', 'Unnamed: 19', '(mm)', 'Unnamed: 6', 'Unnamed: 80', 'Unnamed: 10', 'Unnamed: 17', 'Unnamed: 66', 'Minimum ground clearance', 'parts）(PCS)', 'Unnamed: 34', 'Unnamed: 26', 'Unnamed: 32', 'Unnamed: 30', 'Unnamed: 52', 'Unnamed: 42', 'Unnamed: 12', 'Unnamed: 60', 'Unnamed: 28', 'Unnamed: 22', 'Unnamed: 68', 'Wheel rim Size(Front/Rear)', 'Unnamed: 56', 'Unnamed: 49', 'Unnamed: 71', 'Unnamed: 45', 'Unnamed: 8', 'Unnamed: 2', 'packaging）(SET)', 'Unnamed: 40', 'Tire Size(Front/Rear)', 'Unnamed: 47', 'Unnamed: 87', 'Unnamed: 82', 'Unnamed: 85', 'Unnamed: 54', 'Unnamed: 83', 'Consumption (L/100km)', 'Unnamed: 62', 'Unnamed: 73', 'Unnamed: 78', 'Container loading （SKD', 'Unnamed: 58'}

Cleaned brochure data saved to checkpoints/cleaned_brochure_motorcycle_data.csv


In [57]:
df_exhibition = pd.read_csv('../data/formatted/brochure_exhibition.csv')

# Drop columns with all missing values
df_exhibition_cleaned = df_exhibition.dropna(axis=1, how='all')

# Save the cleaned exhibition dataframe
df_exhibition_cleaned.to_csv('../data/checkpoints/full_exhibition_motorcycle_data.csv', index=False)
print("\nCleaned exhibition data saved to checkpoints/cleaned_exhibition_motorcycle_data.csv")


Cleaned exhibition data saved to checkpoints/cleaned_exhibition_motorcycle_data.csv
